## [Setting] 필요한 라이브러리를 임포트


In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 36.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
import random
import time
import datetime
import os

## [Setting] GPU 관련 설정 확인

In [5]:
n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## [Setting] 파일 경로 설정

In [6]:
os.listdir('drive/MyDrive/주분2주차')

['innocent_유머.csv',
 '최종label1.csv',
 'RNN.ipynb',
 '최종데이터셋.csv',
 'Untitled0.ipynb']

In [7]:
os.chdir('drive/MyDrive/주분2주차')

## 1. 데이터셋 불러오기

In [8]:
data = pd.read_csv("최종데이터셋.csv")

In [10]:
data.head()

,comments,label
0,네 여자친구랑 섹스해도 되냐?,1
1,"이게 청소한 거냐, 이새끼들이 미쳤네, 초임하사 새끼들이 벌써부터 풀려가지고, 니 ...",1
2,"넌 새끼야 왜 똑같이 가르쳤는데 쟤보다 못 하냐, 너 하사 아니가, 하사가 병들을 ...",1
3,"이 미친놈아 너 나 지금 엿먹일려고 작정했냐, 하라는 데만 하면 되는데 왜 이따위로...",1
4,"내가 언제 욕했어, 새끼야.",1


In [ ]:
# # label1 데이터셋 불러오기

# label1 = pd.read_csv("최종label1.csv")
# label1["label"] = np.ones(len(label1))
# label1 = label1[["comments", "label"]]

# # label0 데이터셋 불러오기

# label0 = pd.read_csv("innocent_유머.csv", encoding="cp949")
# label0["label"] = np.zeros(len(label0))
# label0 = label0[["content", "label"]]
# label0.columns = ["comments", "label"]
# label0 = label0.sample(n=len(label1))

# # label1 + label0 데이터셋 결합하기

# data = pd.concat([label1, label0]).reset_index()[["comments", "label"]]

In [11]:
data = data.astype({'label' : 'int64'})

In [12]:
data.head()

,comments,label
0,네 여자친구랑 섹스해도 되냐?,1
1,"이게 청소한 거냐, 이새끼들이 미쳤네, 초임하사 새끼들이 벌써부터 풀려가지고, 니 ...",1
2,"넌 새끼야 왜 똑같이 가르쳤는데 쟤보다 못 하냐, 너 하사 아니가, 하사가 병들을 ...",1
3,"이 미친놈아 너 나 지금 엿먹일려고 작정했냐, 하라는 데만 하면 되는데 왜 이따위로...",1
4,"내가 언제 욕했어, 새끼야.",1


In [13]:
X_train, X_test, y_train, y_test = train_test_split(data["comments"], data["label"], test_size=0.15, stratify=data["label"])

train = pd.DataFrame(X_train)
train["label"] = y_train
train = train.reset_index()[["comments", "label"]]

test = pd.DataFrame(X_test)
test["label"] = y_test
test = test.reset_index()[["comments", "label"]]

## 2. 자연어 데이터 전처리

### (1) 스페셜 토큰 부착

In [14]:
# 스페셜 토큰을 문장 앞뒤로 부착하는 함수

def add_special_token(document):
    added = ["[CLS] " + str(sentence) + " [SEP]" for sentence in document]
    return added

# comments 열에 스페셜 토큰 부착

document_bert = add_special_token(train.comments)

### (2) 토큰화

In [15]:
# BERT로 토큰화

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased',do_lower_case=False)

# BERT를 이용하여 토큰화 처리

tokenized_texts = [tokenizer.tokenize(s) for s in document_bert]

### (3) Padding 추가

In [16]:
# padding 처리하는 함수

MAX_LEN = 128
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen = MAX_LEN, dtype='long', truncating='post', padding='post')

### (4) Attention Mask 생성

In [17]:
# 어텐션 마스크 생성
# 어텐션 마스크? : BERT가 attention 연산 시, padding token에 대해 불필요하게 attention하지 않도록
# 실제 단어가 있는 부분과 padding token이 있는 부분을 구분할 수 있도록 하는 마스크를 만들어 줌.

attention_masks = []

for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

## 3. train_inputs / validation_inputs 로 분리 후 Tensor로 변환

In [18]:
# X_train --> train_inputs, validation_inputs로 분리한다.
# attention mask도 이에 대응하여 분리시킨다. 따라서 random state를 고정하여 train_inputs/validation_inputs의 분할과 동일하게 분할한다.

train_inputs, validation_inputs, train_labels, validation_labels = \
train_test_split(input_ids, train['label'].values, random_state=42, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42,test_size=0.1)

In [19]:
# train을 위한 데이터들을 tensor로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

# validation을 위한 데이터들을 tensor로 변환
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [20]:
train_inputs.shape, train_labels.shape, train_masks.shape

(torch.Size([1928, 128]), torch.Size([1928]), torch.Size([1928, 128]))

In [21]:
BATCH_SIZE = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size = BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

## 4. Test Data 전처리

In [22]:
sentences = test['comments']
sentences = ["[CLS] "+str(sentence)+" [SEP]" for sentence in sentences]
labels = test['label'].values

tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

attention_masks = []
for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

## [Setting] 모델 학습 전 GPU 상태 확인

In [23]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 5. BERT 모델 GPU에 올리고 train

In [24]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
# binary classification이므로 num_labels=2 이다.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [26]:
# transformers에서 제공하는 옵티마이저 중 AdamW를 사용
# 총 훈련 스텝은 이터레이션 * 에폭 수로 설정
# 러닝 레잇 스케쥴러도 transformers에서 제공하는것을 사용
print('schedule start')

#옵티마이저 설정
optimizer = torch.optim.AdamW(model.parameters(),lr = 2e-5, eps = 1e-8)

# 에폭수
epochs = 10

# 총 훈련 스텝
total_steps = len(train_dataloader) * epochs

# lr 조금씩 감소시키는 스케줄러
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

##학습
# accuracy 와 시간 표시함수 정의
# 정확도 계산 함수
print('train start')
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

schedule start
train start


In [27]:
# 데이터로더에서 배치만큼 가져온 후 forward, backward pass를 수행
# gradient update는 명시적으로 하지 않고 위에서 로드한 optimizer를 활용
# 재현을 위해 랜덤시드 고정
# 모든 Epoch를 학습하면 학습이 종료

seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):

    # ========================================
    #               Training
    # ========================================

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask,
                        labels=b_labels)

        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)

        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # 그래디언트 계산 안함
        with torch.no_grad():
            # Forward 수행
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")
print("")


======== Epoch 1 / 10 ========
Training...

  Average training loss: 0.50
  Training epcoh took: 0:00:36

Running Validation...
  Accuracy: 0.80
  Validation took: 0:00:01

======== Epoch 2 / 10 ========
Training...

  Average training loss: 0.25
  Training epcoh took: 0:00:36

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:01

======== Epoch 3 / 10 ========
Training...

  Average training loss: 0.16
  Training epcoh took: 0:00:37

Running Validation...
  Accuracy: 0.91
  Validation took: 0:00:01

======== Epoch 4 / 10 ========
Training...

  Average training loss: 0.11
  Training epcoh took: 0:00:37

Running Validation...
  Accuracy: 0.92
  Validation took: 0:00:02

======== Epoch 5 / 10 ========
Training...

  Average training loss: 0.08
  Training epcoh took: 0:00:38

Running Validation...
  Accuracy: 0.92
  Validation took: 0:00:02

======== Epoch 6 / 10 ========
Training...

  Average training loss: 0.06
  Training epcoh took: 0:00:38

Running Validation...
  Accu

## 6. Test

In [28]:
print('test start')
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

logits_list = []

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)

    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    f1 = f1_score(label_ids.flatten(), np.argmax(logits, axis=1).flatten())
    print("%d ) f1 score of this batch : %.3f"%(step, f1))

    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))
print("test finished!")

test start
0 ) f1 score of this batch : 0.909
1 ) f1 score of this batch : 0.875
2 ) f1 score of this batch : 0.917
3 ) f1 score of this batch : 0.889
4 ) f1 score of this batch : 0.880
5 ) f1 score of this batch : 0.941
6 ) f1 score of this batch : 0.909
7 ) f1 score of this batch : 0.900
8 ) f1 score of this batch : 0.914
9 ) f1 score of this batch : 0.829
10 ) f1 score of this batch : 0.976
11 ) f1 score of this batch : 0.857

Accuracy: 0.90
Test took: 0:00:03
test finished!


## 7. 새로운 문장 TEST

In [29]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [30]:
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)

    # 그래디언트 계산 안함
    with torch.no_grad():
        # Forward 수행
        outputs = model(b_input_ids,
                        token_type_ids=None,
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [31]:
logits = test_sentences(['나는 오늘 감자탕을 먹었어...'])
print(np.exp(logits[0][1]))

9.36379


In [32]:
logits = test_sentences(['죽어 씨발새끼야 제발'])
print(np.exp(logits[0][1]))

29.672081


In [33]:
logits = test_sentences(['항문에서 무지개가 나온다'])
print(np.exp(logits[0][1]))

0.6035235


In [41]:
logits = test_sentences(['사랑해 고마워'])
print(np.exp(logits[0][1]))

0.8145737


## [Setting] 내보내기

In [43]:
!jupyter nbconvert --to html Untitled0.ipynb

[NbConvertApp] Converting notebook Untitled0.ipynb to html
Traceback (most recent call last):
  File "/usr/local/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/jupyter_core/application.py", line 280, in launch_instance
    super().launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 423, in start
    self.convert_notebooks()
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 597, in convert_notebooks
    self.convert_single_notebook(notebook_filename)
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 560, in convert_single_notebook
    output, resources = self.export_single_notebook(
  File "/usr/local/lib/python3.10/dist-packages/nbconvert/nbconvertapp.py", line 488, in e